GitHub<br>
https://github.com/LambdaLabsML/lambda-diffusers<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/lambda_diffusers_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/LambdaLabsML/lambda-diffusers.git


# Commits on Oct 17, 2022
%cd /content/lambda-diffusers
!git checkout 4d4386e5d2ae667d18987b1a505a47efc643e9c9

## ライブラリのインストール

In [ ]:
%cd /content/lambda-diffusers

# for lambdalabs/sd-pokemon-diffusers
!pip install diffusers==0.3.0
!pip install transformers==4.22.1 scipy==1.7.3 ftfy==6.1.1

# for lambda-diffusers
!pip install -r requirements.txt

In [ ]:
raise Exception("Please Restart RUNTIME")

## ライブラリのインポート

In [ ]:
%cd /content/lambda-diffusers

from pathlib import Path
from lambda_diffusers import StableDiffusionImageEmbedPipeline
from PIL import Image
import torch
from torch import autocast

from diffusers import StableDiffusionPipeline


device = "cuda" if torch.cuda.is_available() else "cpu"

# Image to Image

## 学習済みモデルのセットアップ

In [ ]:
pipe = StableDiffusionImageEmbedPipeline.from_pretrained("lambdalabs/sd-image-variations-diffusers")
pipe = pipe.to(device)

## テスト画像のセットアップ

In [ ]:
!rm -rf outputs
!rm -rf inputs
!mkdir outputs inputs

!wget -c https://casie.jp/media/wp-content/uploads/2020/09/Van_Gogh_Self-Portrait_with_Straw_Hat_1887-Detroit.jpg \
      -O inputs/test_01.jpg

## Inference

In [ ]:
input_img_path = "inputs/test_01.jpg"

im = Image.open(input_img_path)
num_samples = 2
image = pipe(num_samples*[im], guidance_scale=3.0)
image = image["sample"]

base_path = Path("outputs")
base_path.mkdir(exist_ok=True, parents=True)
for idx, im in enumerate(image):
  im.save(base_path/f"{idx:06}.jpg")

## 結果表示

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
  h_min = min(im.shape[0] for im in im_list)
  im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation) for im in im_list]
  
  return cv2.hconcat(im_list_resize)

In [ ]:
input_cv_im = cv2.imread(input_img_path)

im_h = None
for i in range(num_samples):
  cv_im = cv2.imread(f"outputs/{i:06}.jpg")
  
  if i == 0:
    im_h = hconcat_resize_min([input_cv_im, cv_im])
  else:
    im_h = cv2.hconcat([im_h, cv_im])

cv2_imshow(im_h)


# Pokemon Text to Image

## 学習済みモデルのセットアップ

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("lambdalabs/sd-pokemon-diffusers", torch_dtype=torch.float16)  
pipe = pipe.to("cuda")

## Prompt

In [ ]:
prompt = "Birth of Venus" #@param {type:"string"}

In [ ]:
# ガイダンススケール
scale = 10
# 生成枚数
n_samples = 3
disable_safety = True

## Inference

In [ ]:
!rm -rf outputs_poke
!mkdir outputs_poke

if disable_safety:
  def null_safety(images, **kwargs):
      return images, False
  pipe.safety_checker = null_safety

with autocast("cuda"):
  images = pipe(n_samples*[prompt], guidance_scale=scale).images

for idx, im in enumerate(images):
  im.save(f"outputs_poke/{idx:06}.png")

## 結果表示

In [ ]:
im_h = None
for i in range(n_samples):
  cv_im = cv2.imread(f"outputs_poke/{i:06}.png")
  
  if i == 0:
    im_h = cv_im
  else:
    im_h = cv2.hconcat([im_h, cv_im])

cv2_imshow(im_h)